In [1]:
import autogluon.core as ag
from autogluon.vision import ImagePredictor

/root/anaconda3/lib/python3.6/site-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.8.0` and `torch==1.9.0+cu102` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


In [2]:
train_dataset, _, test_dataset = ImagePredictor.Dataset.from_folders('https://autogluon.s3.amazonaws.com/datasets/shopee-iet.zip')
print(train_dataset)

data/
├── test/
└── train/
                                                 image  label
0    /root/.gluoncv/datasets/shopee-iet/data/train/...      0
1    /root/.gluoncv/datasets/shopee-iet/data/train/...      0
2    /root/.gluoncv/datasets/shopee-iet/data/train/...      0
3    /root/.gluoncv/datasets/shopee-iet/data/train/...      0
4    /root/.gluoncv/datasets/shopee-iet/data/train/...      0
..                                                 ...    ...
795  /root/.gluoncv/datasets/shopee-iet/data/train/...      3
796  /root/.gluoncv/datasets/shopee-iet/data/train/...      3
797  /root/.gluoncv/datasets/shopee-iet/data/train/...      3
798  /root/.gluoncv/datasets/shopee-iet/data/train/...      3
799  /root/.gluoncv/datasets/shopee-iet/data/train/...      3

[800 rows x 2 columns]


In [3]:
predictor = ImagePredictor()
# since the original dataset does not provide validation split, the `fit` function splits it randomly with 90/10 ratio
predictor.fit(train_dataset, hyperparameters={'epochs': 2})  # you can trust the default config, we reduce the # epoch to save some build time

INFO:root:`time_limit=auto` set to `time_limit=7200`.
INFO:root:Reset labels to [0, 1, 2, 3]
INFO:gluoncv.auto.tasks.image_classification:Randomly split train_data into train[721]/validation[79] splits.
INFO:gluoncv.auto.tasks.image_classification:Starting fit without HPO
INFO:ImageClassificationEstimator:modified configs(<old> != <new>): {
INFO:ImageClassificationEstimator:root.train.batch_size 128 != 16
INFO:ImageClassificationEstimator:root.train.num_workers 4 != 56
INFO:ImageClassificationEstimator:root.train.rec_train ~/.mxnet/datasets/imagenet/rec/train.rec != auto
INFO:ImageClassificationEstimator:root.train.rec_val_idx ~/.mxnet/datasets/imagenet/rec/val.idx != auto
INFO:ImageClassificationEstimator:root.train.rec_train_idx ~/.mxnet/datasets/imagenet/rec/train.idx != auto
INFO:ImageClassificationEstimator:root.train.data_dir  ~/.mxnet/datasets/imagenet != auto
INFO:ImageClassificationEstimator:root.train.rec_val   ~/.mxnet/datasets/imagenet/rec/val.rec != auto
INFO:ImageClassifi

In [4]:
fit_result = predictor.fit_summary()
print('Top-1 train acc: %.3f, val acc: %.3f' %(fit_result['train_acc'], fit_result['valid_acc']))

Top-1 train acc: 0.451, val acc: 0.620


In [5]:
image_path = test_dataset.iloc[0]['image']
result = predictor.predict(image_path)
print(result)

0    1
Name: label, dtype: int64


In [6]:
proba = predictor.predict_proba(image_path)
print(proba)

          0         1         2         3
0  0.144171  0.496033  0.114388  0.245408


In [7]:
bulk_result = predictor.predict(test_dataset)
print(bulk_result)

0     1
1     1
2     1
3     2
4     1
     ..
75    3
76    3
77    3
78    0
79    3
Name: label, Length: 80, dtype: int64


In [8]:
image_path = test_dataset.iloc[0]['image']
feature = predictor.predict_feature(image_path)
print(feature)

                                       image_feature
0  [0.0059656673, 0.0, 0.2278668, 0.48162577, 0.3...


/root/anaconda3/lib/python3.6/site-packages/mxnet/gluon/block.py:682: UserWarning: Parameter resnetv1b_dense0_bias, resnetv1b_dense0_weight is not used by any computation. Is this intended?
  out = self.forward(*args)


In [9]:
test_acc, _ = predictor.evaluate(test_dataset)
print('Top-1 test acc: %.3f' % test_acc)

Top-1 test acc: 0.625


In [10]:
filename = 'predictor.ag'
predictor.save(filename)
predictor_loaded = ImagePredictor.load(filename)
# use predictor_loaded as usual
result = predictor_loaded.predict(image_path)
print(result)

0    1
Name: label, dtype: int64
